# Step 1 to create GOES data timeseries

- Creates files with lists of all GOES files in 1 day

- Run this before creating GOES timeseries data

- Set start date

- Set number of days to increment through (does 1 day at a time)

This primarily supports creating the historical GOES data. This has to access 288 files per day, and with all those requests, there is risk of failure. Having a static list of filenames to go through was a quick solution to be able to make sure we started again at the same place we ended when the program died.

You can check what data is available here: [https://noaa-goes16.s3.amazonaws.com/index.html#ABI-L2-MCMIPC/]()



In [ ]:
#create goes data
from goes2go import GOES
import xarray as xr
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

#################  SET THESE VALUES ######################

#start_year, start_month, start_day = 2023,12,6
#number_days_to_process = 9

##########################################################

#import configuration location and filepath
from myconfig import *
output_path = output_path_fname

def get_start_end_time(fname):
    #goes filenames structure https://geonetcast.wordpress.com/2017/04/27/goes-16-file-naming-convention/
    #use filename to find start/end times for data
    tem = str(fname).split('/')
    tem2,i = tem[5],25
    dt_start = datetime.strptime(tem2[i:i+13], '%Y%j%h%M%S')
    tem2,i = tem[5],41
    dt_end = datetime.strptime(tem2[i:i+13], '%Y%j%h%M%S')
    return dt_start,dt_end


In [ ]:
now = datetime.now()
end_time = now.strftime("%Y-%m-%d %H:%M")
yesterday = now - timedelta(hours=24)
start_time = yesterday.strftime("%Y-%m-%d %H:%M")

for incr in range(0,number_days_to_process):

    date_start = datetime(start_year, start_month, start_day) + timedelta(days=incr)
    date_end = date_start + timedelta(days=1)  
    start_time = date_start.strftime("%Y%m%d")
    end_time = date_end.strftime("%Y%m%d")
    
    print(start_time,end_time)
    if incr==3:
        continue
    # Open the GOES-R image
    G = GOES(satellite=16, product="ABI-L2-MCMIPC", domain='C')  #ABI-L2-DMWVC ABI-L2-DMWC not at all points #
    # Produce a pandas DataFrame of the available files in a time range
    df = G.df(start=start_time, end=end_time)
    print(len(df), df.file[0])
    # Save DataFrame to a CSV file
    files_fname = output_path+'goes_filenames_test_'+start_time+'-'+end_time+'.csv'
    df.to_csv(files_fname)


In [ ]:
#FIX FOR 2021 where day 243 hour 16 is missing on cloud

date_start = datetime(start_year, start_month, start_day) + timedelta(days=incr)
date_end = date_start + timedelta(hours=15)  
start_time = date_start.strftime("%Y%m%d %H:00")
end_time = date_end.strftime("%Y%m%d %H:59")
    
print(start_time,end_time)
    
    # Open the GOES-R image
G = GOES(satellite=16, product="ABI-L2-MCMIPC", domain='C')  #ABI-L2-DMWVC ABI-L2-DMWC not at all points #
    # Produce a pandas DataFrame of the available files in a time range
df = G.df(start=start_time, end=end_time)

date_start = datetime(start_year, start_month, start_day) + timedelta(days=incr) 
date_start = date_start + timedelta(hours=17)  
date_start2 = datetime(start_year, start_month, start_day) + timedelta(days=incr)
date_end = date_start2 + timedelta(days=1)
start_time = date_start.strftime("%Y%m%d %H:00")
end_time = date_end.strftime("%Y%m%d %H:00")
    
print(start_time,end_time)
    
    # Open the GOES-R image
G = GOES(satellite=16, product="ABI-L2-MCMIPC", domain='C')  #ABI-L2-DMWVC ABI-L2-DMWC not at all points #
    # Produce a pandas DataFrame of the available files in a time range
df2 = G.df(start=start_time, end=end_time)
df3 = pd.concat([df, df2], ignore_index=True)

date_start = datetime(start_year, start_month, start_day) + timedelta(days=incr)
date_end = date_start + timedelta(days=1)  
start_time = date_start.strftime("%Y%m%d")
end_time = date_end.strftime("%Y%m%d")

files_fname = output_path+'goes_filenames_test_'+start_time+'-'+end_time+'.csv'
df3.to_csv(files_fname)
